In [1]:
import os
import openai
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import docx

load_dotenv(dotenv_path='.env')
openai_api_key = os.getenv("OPENAI_API_KEY")

#pip install langchain==0.1.1
#pip install langchain_community==0.0.13
#pip install langchain_openai
#pip install tiktoken==0.4.0
#pip install pinecone-client==3.1.0
#pip install pyarrow --only-binary :all:
#pip install pinecone-datasets==0.7.0
#pip install pinecone-notebooks==0.1.1
#pip install langchainhub


In [2]:
# https://github.com/infoslack/youtube/blob/main/embeddings/Pinecone.ipynb
# https://www.youtube.com/watch?v=iw2TeYESnTk
# Como utilizar OpenAI Embeddings e Pinecone para busca semântica

from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key, model="text-embedding-3-small")

In [3]:
docs_path = "dados"
text_chunks = []
texto_completo = ""
for f_name in os.listdir(docs_path):
    if f_name.endswith('.docx'):
        print(f_name)
        doc_path = os.path.join(docs_path, f_name)
        doc = docx.Document(doc_path)
        for p in doc.paragraphs:
            text_chunks.append(p.text)
            texto_completo = texto_completo + p.text

lei9279.docx


In [4]:
# remove all chunks shorter than 10 words and strip the rest
# text_chunks = [string.strip().strip('\n') for string in text_chunks if len(string.split()) >= 1]
#text_chunks = [texto_completo.strip().split('#')]
text_chunks = [chunk.strip() for chunk in texto_completo.split('#')]
print(text_chunks)

['LEI Nº 9.279, DE 14 DE MAIO DE 1996Regula direitos e obrigações relativos à propriedade industrial.        O  PRESIDENTE DA REPÚBLICA Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:DISPOSIÇÕES PRELIMINARES        Art. 1º Esta Lei regula direitos e obrigações relativos à propriedade industrial.        Art. 2º A proteção dos direitos relativos à propriedade industrial, considerado o seu interesse social e o desenvolvimento tecnológico e econômico do País, efetua-se mediante:        I - concessão de patentes de invenção e de modelo de utilidade;        II - concessão de registro de desenho industrial;        III - concessão de registro de marca;        IV - repressão às falsas indicações geográficas; e        V - repressão à concorrência desleal.        VI – concessão de registro para jogos eletrônicos.    (Incluído pela Lei nº 14.852, de 2024)        Art. 3º Aplica-se também o disposto nesta Lei:        I - ao pedido de patente ou de registro proveniente do ex

In [5]:
chunks_with_embeddings = []
for chunk in text_chunks:
    embedding = embeddings_model.embed_query(chunk)
    chunks_with_embeddings.append({"text": chunk, "embedding": embedding})

In [6]:
# https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/langchain-retrieval-augmentation.ipynb#scrollTo=EQ3IKSGxDkY9
# https://app.pinecone.io/
from dotenv import load_dotenv
import os, time
from pinecone import Pinecone
from pinecone import ServerlessSpec

pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)
index_name = 'my-index'
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
pc.create_index(
    name='my-index', 
    dimension=1536, 
    metric='euclidean',
    spec = spec
)

# host: https://my-index-imi3zbc.svc.aped-4627-b74a.pinecone.io

C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)
print("Inicializado")

Inicializado


In [8]:
index = pc.Index(index_name)
# wait a moment for connection
time.sleep(1)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [20]:
# create index

index_name = 'livro-python'
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
pc.create_index(
    name='livro-python', 
    dimension=1536, 
    metric='euclidean',
    spec = spec
)
# connect to index
index = pc.Index(index_name)

# host: https://livro-python-imi3zbc.svc.aped-4627-b74a.pinecone.io

In [10]:
len(chunks_with_embeddings)

77

In [11]:
chunks_with_embeddings[60]

{'text': 'CAPÍTULO IXDA PATENTE DE INTERESSE DA DEFESA NACIONAL        Art. 75. O pedido de patente originário do Brasil cujo objeto interesse à defesa nacional será processado em caráter sigiloso e não estará sujeito às publicações previstas nesta Lei.       (Regulamento)        § 1º O INPI encaminhará o pedido, de imediato, ao órgão competente do Poder Executivo para, no prazo de 60 (sessenta) dias, manifestar-se sobre o caráter sigiloso. Decorrido o prazo sem a manifestação do órgão competente, o pedido será processado normalmente.        § 2º É vedado o depósito no exterior de pedido de patente cujo objeto tenha sido considerado de interesse da defesa nacional, bem como qualquer divulgação do mesmo, salvo expressa autorização do órgão competente.        § 3º A exploração e a cessão do pedido ou da patente de interesse da defesa nacional estão condicionadas à prévia autorização do órgão competente, assegurada indenização sempre que houver restrição dos direitos do depositante ou do 

In [12]:
i=0
batch_size = 1
data_batch = chunks_with_embeddings[i: i+batch_size]
data_batch

[{'text': 'LEI Nº 9.279, DE 14 DE MAIO DE 1996Regula direitos e obrigações relativos à propriedade industrial.        O  PRESIDENTE DA REPÚBLICA Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:DISPOSIÇÕES PRELIMINARES        Art. 1º Esta Lei regula direitos e obrigações relativos à propriedade industrial.        Art. 2º A proteção dos direitos relativos à propriedade industrial, considerado o seu interesse social e o desenvolvimento tecnológico e econômico do País, efetua-se mediante:        I - concessão de patentes de invenção e de modelo de utilidade;        II - concessão de registro de desenho industrial;        III - concessão de registro de marca;        IV - repressão às falsas indicações geográficas; e        V - repressão à concorrência desleal.        VI – concessão de registro para jogos eletrônicos.    (Incluído pela Lei nº 14.852, de 2024)        Art. 3º Aplica-se também o disposto nesta Lei:        I - ao pedido de patente ou de registro provenie

In [13]:
i_end = min(i+batch_size, len(chunks_with_embeddings))
print(i_end)

1


In [14]:
text_batch = [item["text"] for item in data_batch]
text_batch

['LEI Nº 9.279, DE 14 DE MAIO DE 1996Regula direitos e obrigações relativos à propriedade industrial.        O  PRESIDENTE DA REPÚBLICA Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:DISPOSIÇÕES PRELIMINARES        Art. 1º Esta Lei regula direitos e obrigações relativos à propriedade industrial.        Art. 2º A proteção dos direitos relativos à propriedade industrial, considerado o seu interesse social e o desenvolvimento tecnológico e econômico do País, efetua-se mediante:        I - concessão de patentes de invenção e de modelo de utilidade;        II - concessão de registro de desenho industrial;        III - concessão de registro de marca;        IV - repressão às falsas indicações geográficas; e        V - repressão à concorrência desleal.        VI – concessão de registro para jogos eletrônicos.    (Incluído pela Lei nº 14.852, de 2024)        Art. 3º Aplica-se também o disposto nesta Lei:        I - ao pedido de patente ou de registro proveniente do ex

In [15]:
ids_batch = [str(n) for n in range(i, i_end)]
print(ids_batch)

['0']


In [16]:
embeds = [item["embedding"] for item in data_batch]
embeds

[[0.05958885699510574,
  0.03779951110482216,
  -0.010476039722561836,
  0.025710202753543854,
  0.018348384648561478,
  0.040617626160383224,
  -0.0170005913823843,
  -0.01889975555241108,
  -0.0648166686296463,
  -0.015009530819952488,
  0.037288982421159744,
  -0.024525776505470276,
  -0.028855057433247566,
  -0.03941278159618378,
  0.015326058492064476,
  -0.0027976948767900467,
  0.004656017757952213,
  -0.001675809035077691,
  0.05423852056264877,
  -0.005092519335448742,
  0.04860229045152664,
  0.05628063529729843,
  -0.04214921221137047,
  -0.0013682156568393111,
  -0.0037345141172409058,
  -0.04700944200158119,
  -0.005243124905973673,
  -0.021462608128786087,
  -0.008214399218559265,
  0.036676350980997086,
  0.020094390958547592,
  -0.011333727277815342,
  -0.029426848515868187,
  -0.02330050989985466,
  -0.011405200697481632,
  0.006105917971581221,
  -0.027466420084238052,
  -0.03110138140618801,
  -0.0014881897950544953,
  0.05166545882821083,
  0.004216963425278664,
  -

In [17]:
meta = [{"text": text_batch} for text_batch in zip(text_batch)]
meta

[{'text': ('LEI Nº 9.279, DE 14 DE MAIO DE 1996Regula direitos e obrigações relativos à propriedade industrial.        O  PRESIDENTE DA REPÚBLICA Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:DISPOSIÇÕES PRELIMINARES        Art. 1º Esta Lei regula direitos e obrigações relativos à propriedade industrial.        Art. 2º A proteção dos direitos relativos à propriedade industrial, considerado o seu interesse social e o desenvolvimento tecnológico e econômico do País, efetua-se mediante:        I - concessão de patentes de invenção e de modelo de utilidade;        II - concessão de registro de desenho industrial;        III - concessão de registro de marca;        IV - repressão às falsas indicações geográficas; e        V - repressão à concorrência desleal.        VI – concessão de registro para jogos eletrônicos.    (Incluído pela Lei nº 14.852, de 2024)        Art. 3º Aplica-se também o disposto nesta Lei:        I - ao pedido de patente ou de registro proveni

In [18]:
for item in meta:
    if isinstance(item.get('text'), tuple):
        item['text'] = str(item['text'])

In [19]:
to_upsert = zip(ids_batch, embeds, meta)
to_upsert
index.upsert(vectors=list(to_upsert))

{'upserted_count': 1}

In [21]:
# process everything in batches of 64
batch_size = 1

for i in range(0, len(chunks_with_embeddings), batch_size):
    data_batch = chunks_with_embeddings[i: i+batch_size]
    
    # set end position of batch
    i_end = min(i+batch_size, len(chunks_with_embeddings))
    
    # get batch meta
    text_batch = [item["text"] for item in data_batch]
    
    # get ids
    ids_batch = [str(n) for n in range(i, i_end)]
    
    # get embeddings
    embeds = [item["embedding"] for item in data_batch]
    
    # prepare metadata and upsert batch
    meta = [{"text": text_batch} for text_batch in zip(text_batch)]

    for item in meta:
        if isinstance(item.get('text'), tuple):
            item['text'] = str(item['text'])

    to_upsert = zip(ids_batch, embeds, meta)

    index.upsert(vectors=list(to_upsert)) # upsert to Pinecone

In [22]:
# https://platform.openai.com/docs/guides/embeddings/embedding-models
from openai import OpenAI

def search_docs(query):
    client = OpenAI(api_key=openai_api_key)
    xq = client.embeddings.create(input=query, model="text-embedding-3-small").data[0].embedding
    res = index.query(vector=[xq], top_k=10, include_metadata=True)
    #chosen_text = []
    #for match in res["matches"]:
    #    chosen_text = match["metadata"]
    return res["matches"]

In [ ]:
query="Pedido relacionado à segurança nacional é processado em sigilo ?"
client = OpenAI(api_key=openai_api_key)
xq = client.embeddings.create(input=query, model="text-embedding-3-small").data[0].embedding
print(xq)

In [24]:
matches = search_docs("Pedido relacionado à segurança nacional é processado em sigilo ?")
for match in matches:
    print(f"{match['score']:.2f}: {match['metadata']}")

0.70: {'text': "('CAPÍTULO IXDA PATENTE DE INTERESSE DA DEFESA NACIONAL        Art. 75. O pedido de patente originário do Brasil cujo objeto interesse à defesa nacional será processado em caráter sigiloso e não estará sujeito às publicações previstas nesta Lei.       (Regulamento)        § 1º O INPI encaminhará o pedido, de imediato, ao órgão competente do Poder Executivo para, no prazo de 60 (sessenta) dias, manifestar-se sobre o caráter sigiloso. Decorrido o prazo sem a manifestação do órgão competente, o pedido será processado normalmente.        § 2º É vedado o depósito no exterior de pedido de patente cujo objeto tenha sido considerado de interesse da defesa nacional, bem como qualquer divulgação do mesmo, salvo expressa autorização do órgão competente.        § 3º A exploração e a cessão do pedido ou da patente de interesse da defesa nacional estão condicionadas à prévia autorização do órgão competente, assegurada indenização sempre que houver restrição dos direitos do depositant

In [23]:
def construct_prompt(query):
    matches = search_docs(query)
    
    chosen_text = []
    for match in matches:
        chosen_text.append(match["metadata"]["text"])
        
    prompt = """Responda à pergunta com a maior sinceridade possível usando o contexto abaixo e,
                se a resposta não for dentro do contexto, diga apenas: eu não sei."""
    prompt += "\n\n"
    prompt += "Contexto: " + "\n".join(str(text) for text in chosen_text)
    prompt += "\n\n"
    prompt += "Pergunta: " + query
    prompt += "\n"
    prompt += "Resposta: "
    return prompt

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

query = "Pedido relacionado à segurança nacional é processado em sigilo ?"
#query = "Quando o modelo de utilidade é dotado de ato inventivo ?"
template = construct_prompt(query)
prompt = ChatPromptTemplate.from_template(template=template)

llm = ChatOpenAI(openai_api_key=openai_api_key)
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"query":query})
print(response)

Sim, o pedido relacionado à segurança nacional é processado em sigilo, conforme o Art. 75 da Lei de Patentes.


In [26]:
query = "quem descobriu o Brasil ?"
template = construct_prompt(query)
prompt = ChatPromptTemplate.from_template(template=template)

llm = ChatOpenAI(openai_api_key=openai_api_key)
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"query":query})
print(response)

Eu não sei.


In [ ]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
index_name = 'livro-python'
index = pc.Index(index_name)

def search_docs(query):
    client = OpenAI(api_key=openai_api_key)
    xq = client.embeddings.create(input=query, model="text-embedding-3-small").data[0].embedding
    res = index.query(vector=[xq], top_k=77, include_metadata=True)
    #chosen_text = []
    #for match in res["matches"]:
    #    chosen_text = match["metadata"]
        
    return res["matches"]

matches = search_docs("Pedido relacionado à segurança nacional é processado em sigilo ?")
for match in matches:
    print(f"{match['score']:.2f}: {match['metadata']}")